In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers.core import Flatten
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

Using TensorFlow backend.
/home/p4/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df = pd.read_csv('2015_sort.csv')
df.head()

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,04-12-15,MJ1311YNG6KXVA,Hitachi HDS5C3030ALA630,3.000000e+12,0,100.0,0.0,134.0,112.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27-01-15,9XW03B5R,ST31500541AS,1.500000e+12,1,118.0,189882022.0,NaN,NaN,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13-04-15,JK1160YAHP0HTT,Hitachi HDS722020ALA330,2.000000e+12,1,100.0,0.0,100.0,0.0,131.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18-10-15,PL1331LAGR6BMH,HGST HMS5C4040ALE640,4.000000e+12,0,100.0,0.0,133.0,104.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13-04-15,WD-WMC1T0751577,WDC WD30EFRX,3.000000e+12,1,200.0,55.0,NaN,NaN,175.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_norm = df[['smart_1_normalized', 'smart_3_normalized', 'smart_4_normalized', 'smart_5_normalized',
           'smart_7_normalized','smart_12_normalized', 'smart_194_normalized', 
           'smart_197_normalized', 'smart_198_normalized', 'smart_199_normalized']]
df_norm.sample(n=5)

,smart_1_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_12_normalized,smart_194_normalized,smart_197_normalized,smart_198_normalized,smart_199_normalized
1310,200.0,100.0,100.0,200.0,100.0,100.0,120.0,200.0,100.0,200.0
600,100.0,122.0,100.0,100.0,100.0,100.0,166.0,100.0,100.0,200.0
903,118.0,92.0,100.0,100.0,89.0,100.0,23.0,100.0,100.0,200.0
2158,200.0,177.0,100.0,200.0,100.0,100.0,124.0,200.0,100.0,200.0
1043,117.0,96.0,100.0,100.0,87.0,100.0,30.0,100.0,100.0,200.0


In [6]:
X = df_norm.dropna(0,thresh =10).dropna(1)
y = df['failure'].values
y = np.delete(y,1)

In [38]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=123)

In [112]:
model = Sequential()
model.add(Dense(32, input_dim=10, kernel_initializer='normal', activation='relu'))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal',activation ="relu"))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [113]:
print(model.summary())
print(x_train.shape[1])
print(y_train.shape)
print(x_train.shape)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_32 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 17        
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________
None
10
(2284,)
(2284, 10)


In [119]:
model.fit(x_train, y_train,  batch_size=50)

Epoch 1/1
2284/2284 [==============================] - 0s 85us/step - loss: 0.6958 - acc: 0.5123


In [120]:
results = model.evaluate(x_test, y_test, verbose=1)
model.metrics_names

572/572 [==============================] - 0s 83us/step


['loss', 'acc']

In [121]:
results

[0.6963774928679833, 0.5384615378363149]

In [122]:
from sklearn.metrics import classification_report,confusion_matrix
y_pred = model.predict_classes(x_test)
print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test, y_pred))
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy",accuracy)

[[199  84]
 [180 109]]


In [123]:
print("Trained: ",model)
print("Precision:",precision_score(y_test,y_pred))
print("Recall:",recall_score(y_test,y_pred))
print("Accuracy:",accuracy_score(y_test,y_pred))
print("F1-Score",f1_score(y_test,y_pred))

('Trained: ', <keras.engine.sequential.Sequential object at 0x7f086da9dad0>)
('Precision:', 0.5647668393782384)
('Recall:', 0.3771626297577855)
('Accuracy:', 0.5384615384615384)
('F1-Score', 0.45228215767634855)
